- https://github.com/Arshad221b/Sign-Language-Recognition/blob/master/ASLwithCNN
- https://data-flair.training/blogs/sign-language-recognition-python-ml-opencv/

In [49]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import Conv2D,MaxPooling2D, Dense,Flatten, Dropout
import matplotlib.pyplot as plt

In [9]:
# Loading the dataset
import pandas as pd
import numpy as np

training_set = pd.read_csv('./data/sign_mnist_train.csv')
test_set = pd.read_csv('./data/sign_mnist_test.csv')

In [39]:
SIZE = (28, 28)
CLASSES = dict(zip(range(0, 26), list(map(chr, range(97, 123))))) # labels from A to Z

In [30]:
def preprocess_data(data):
    global SIZE
    labels, images = data[0:, 0], np.array([np.reshape(i[1:], SIZE) for i in data])
    return images, labels

In [42]:
X_train, y_train = preprocess_data(training_set.to_numpy())
X_test, y_test = preprocess_data(test_set.to_numpy())

y_train = np.array(y_train).reshape(-1)
y_test = np.array(y_test).reshape(-1)

y_train = np.eye(len(CLASSES))[y_train]
y_test = np.eye(len(CLASSES))[y_test]

In [43]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((27455, 28, 28), (27455, 26), (7172, 28, 28), (7172, 26))

In [48]:
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

In [50]:
classifier = Sequential()
classifier.add(Conv2D(filters=8, kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(28,28,1),activation='relu', data_format='channels_last'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(filters=16, kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(MaxPooling2D(pool_size=(4,4)))
classifier.add(Dense(128, activation='relu'))
classifier.add(Flatten())
classifier.add(Dense(26, activation='softmax'))
classifier.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.fit(X_train, y_train, epochs=50, batch_size=100)

Epoch 1/50
275/275 [==============================] - 16s 42ms/step - loss: 2.6517 - accuracy: 0.4597
Epoch 2/50
275/275 [==============================] - 11s 42ms/step - loss: 0.5340 - accuracy: 0.8169
Epoch 3/50
275/275 [==============================] - 11s 40ms/step - loss: 0.3334 - accuracy: 0.8831
Epoch 4/50
275/275 [==============================] - 11s 40ms/step - loss: 0.2488 - accuracy: 0.9127
Epoch 5/50
275/275 [==============================] - 11s 40ms/step - loss: 0.1946 - accuracy: 0.9323
Epoch 6/50
275/275 [==============================] - 11s 41ms/step - loss: 0.1688 - accuracy: 0.9425
Epoch 7/50
275/275 [==============================] - 11s 41ms/step - loss: 0.1397 - accuracy: 0.9529
Epoch 8/50
275/275 [==============================] - 11s 41ms/step - loss: 0.1295 - accuracy: 0.9557
Epoch 9/50
275/275 [==============================] - 11s 42ms/step - loss: 0.1159 - accuracy: 0.9610
Epoch 10/50
275/275 [==============================] - 11s 40ms/step - loss: 0.104

In [60]:
accuracy = classifier.evaluate(x=X_test,y=y_test,batch_size=32)
print("Accuracy: ",accuracy[1]*100)

225/225 [==============================] - 2s 7ms/step - loss: 0.2886 - accuracy: 0.9030
Accuracy:  90.2955949306488
